Importing necessary libraries

In [79]:
import pandas as pd

Map continent data with yield.csv

In [80]:
yield_data = pd.read_csv('processed_data/processed_yield.csv')

continent_data = pd.read_csv('processed_data/countries_by_continent_sorted.csv')

combined_yield = pd.merge(yield_data, continent_data, how='left', left_on='Area', right_on='Country')

combined_yield = combined_yield.drop(columns=['Country']).rename(columns={'Continent': 'Country_Continent'})

combined_yield.to_csv('processed_data/yield_continent.csv', index=False)

Map pesticides data with the updated yield

In [81]:
yield_data = pd.read_csv('processed_data/yield_continent.csv')

pesticides_data = pd.read_csv('processed_data/processed_pesticides.csv')

combined_yield = pd.merge(yield_data, pesticides_data, how='left', on=['Area', 'Year'])

combined_yield.to_csv('processed_data/yield_continent_pesticides.csv', index=False)


Map rainfall data with the updated yield

In [82]:
yield_data = pd.read_csv('processed_data/yield_continent_pesticides.csv')
 
rainfall_data = pd.read_csv('processed_data/processed_rainfall.csv')
 
combined_yield = pd.merge(yield_data, rainfall_data, how='left', on=['Area', 'Year'])
 
combined_yield.to_csv('processed_data/yield_continent_pesticides_rainfall.csv', index=False)


Map temperature data with the updated yield

In [83]:
yield_data = pd.read_csv('processed_data/yield_continent_pesticides_rainfall.csv')
 
temperature_data = pd.read_csv('processed_data/processed_temp.csv')

combined_yield = pd.merge(yield_data, temperature_data, how='left', left_on=['Area', 'Year'], right_on=['country', 'year'])

combined_yield = combined_yield.drop(columns=['country', 'year'])

combined_yield.to_csv('processed_data/combined_raw_data.csv', index=False)


Remove unecessary columns

In [84]:
file_path = 'processed_data/combined_raw_data.csv'
 
columns_to_delete = ['Domain Code', 'Domain_x', 'Area Code', 'Element_x', 'Element Code', 'Year Code', 'Domain_y', 'Element_y', ]

df = pd.read_csv(file_path)

df = df.drop(columns=columns_to_delete, errors='ignore')

df.to_csv(file_path, index=False)

delete rows that do not have values for the following columns 'Item_y', 'Unit_y', 'Value_y'

In [85]:

file_path = 'processed_data/combined_raw_data.csv'

columns_to_delete = ['Item_y', 'Unit_y', 'Value_y', 'Country_Continent']

df = pd.read_csv(file_path)

records_before = len(df)

df = df.dropna(subset=columns_to_delete, how='any')

records_after = len(df)

df.to_csv(file_path, index=False)
 
n = records_before - records_after
 
print(f"deleted: {n} records")


deleted: 4180 records


count the number of records per continent

In [86]:
# before renaming the column
# file_path = 'processed_data/combined_raw_data.csv'
# df = pd.read_csv(file_path)

# result = df.groupby(['Country_Continent', 'Item_x']).size().reset_index(name='Count')

# print(result)

# file_path = 'processed_data/combined_raw_data.csv'
# df = pd.read_csv(file_path)

# result = df.groupby(['Continent', 'Crop']).size().reset_index(name='Count')

# print(result)


check the rows that do have only records for year range 1992-2013

In [87]:

file_path = 'processed_data/combined_raw_data.csv'
df = pd.read_csv(file_path)

# column_to_check = 'avg_temp'
column_to_check = 'average_rain_fall_mm_per_year'

df['Year'] = pd.to_numeric(df['Year'], errors='coerce')

rows_to_delete = df[(df['Year'] >= 1992) & (df['Year'] <= 2013) & (df[column_to_check].isnull())].index

df.drop(rows_to_delete, inplace=True)

df.to_csv(file_path, index=False)


fill in the values where there are single missing values for average_rain_fall_mm_per_year

In [88]:
def fill_empty_with_mean(csv_file_path): 
    df = pd.read_csv(csv_file_path)
 
    for i in range(1, len(df) - 1):
        if pd.isna(df.at[i, 'average_rain_fall_mm_per_year']): 
            mean_value = (df.at[i-1, 'average_rain_fall_mm_per_year'] + df.at[i+1, 'average_rain_fall_mm_per_year']) / 2
 
            df.at[i, 'average_rain_fall_mm_per_year'] = mean_value
 
    df.to_csv(csv_file_path, index=False)
 
fill_empty_with_mean('processed_data/combined_raw_data.csv')

managing final column operations(delete, rename, reordering)

In [89]:
file_path = 'processed_data/combined_raw_data.csv'

rename_columns = {
    # 'old_column_name': 'new_column_name',
    'Area': 'Country', 
    'Item_x': "Crop",
    'Value_x': 'Quantity(hg/ha)',
    'Country_Continent': 'Continent',
    'Value_y': 'Pesticide use(toai)',
    'average_rain_fall_mm_per_year': 'Rainfall(mm)',
    'avg_temp': 'Temperature(C)', 
}
 
reorder_columns = ['Continent', 'Country', 'Year', 'Temperature(C)', 'Rainfall(mm)', 'Pesticide use(toai)', 'Crop', 'Quantity(hg/ha)']
 
# delete_columns = ['column_to_delete1', 'column_to_delete2', 'column_to_delete3']

df = pd.read_csv(file_path)

df.rename(columns=rename_columns, inplace=True)
 
df = df[reorder_columns]
 
df.to_csv(file_path, index=False)

import os 
os.remove('processed_data/yield_continent_pesticides.csv')
os.remove('processed_data/yield_continent.csv')
os.remove('processed_data/yield_continent_pesticides_rainfall.csv')
os.remove('processed_data/processed_temp.csv') 
os.remove('processed_data/processed_pesticides.csv')
os.remove('processed_data/processed_rainfall.csv')
os.remove('processed_data/processed_yield.csv') 




In [8]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# Load your data from the CSV file
file_path = 'processed_data/combined_raw_data.csv'
yield_df = pd.read_csv(file_path)

# Select columns to one-hot encode
columns_to_encode = ['Country', 'Crop', 'Pesticide use(toai)']

# Perform one-hot encoding
encoder = OneHotEncoder(drop='first', sparse=False)
encoded_features = pd.DataFrame(encoder.fit_transform(yield_df[columns_to_encode]),
                                columns=encoder.get_feature_names_out(columns_to_encode))

# Concatenate the encoded features with the original dataframe
yield_df_encoded = pd.concat([yield_df, encoded_features], axis=1)

# Drop the original columns that were encoded
yield_df_encoded.drop(columns=columns_to_encode, inplace=True)

# Separate features and labels
features = yield_df_encoded.loc[:, yield_df_encoded.columns != 'Quantity(hg/ha)']
label = yield_df['Quantity(hg/ha)']

# Display the head of the features dataframe
print(features.head()) 
features = features.drop(['Year'], axis=1)
features = features.drop(['Continent'], axis=1)
features.to_csv("urmom.csv", index=False)
features.info()


/Users/sandu/Library/Python/3.9/lib/python/site-packages/sklearn/preprocessing/_encoders.py:808: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


  Continent  Year  Temperature(C)  Rainfall(mm)  Country_Algeria  \
0    Europe  1992           16.06        1485.0              0.0   
1    Europe  1993           16.05        1485.0              0.0   
2    Europe  1994           16.96        1485.0              0.0   
3    Europe  1995           15.67        1485.0              0.0   
4    Europe  1996           15.64        1485.0              0.0   

   Country_Angola  Country_Antigua and Barbuda  Country_Argentina  \
0             0.0                          0.0                0.0   
1             0.0                          0.0                0.0   
2             0.0                          0.0                0.0   
3             0.0                          0.0                0.0   
4             0.0                          0.0                0.0   

   Country_Armenia  Country_Australia  ...  Pesticide use(toai)_214725.0  \
0              0.0                0.0  ...                           0.0   
1              0.0      